In [2]:
import numpy as np
import pickle
import os
import time
import sys
import matplotlib.pyplot as plt
import cv2 as cv
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler


LOAD_FROM_PICKLE = True

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' # or any {‘0’, ‘1’, ‘2’}
os.environ['AUTOGRAPH_VERBOSITY'] = '0'

import warnings
warnings.simplefilter('ignore', FutureWarning)
warnings.simplefilter('ignore', RuntimeWarning)

USE_GPU = True

In [3]:
# The kernel needs to be restarted before changing this setting to take effect

if USE_GPU:
    os.environ["CUDA_VISIBLE_DEVICES"] = "0"
else:
    os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [4]:
# Tensorflow import needs to be after setting the CUDA_VISIBLE_DEVICES

import tensorflow as tf
from tensorflow import keras
from tensorflow.python.keras.layers import Dense, Activation, Input
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.layers import concatenate, Add
from tensorflow.python.keras import backend as K
from keras.utils.vis_utils import plot_model

# print Tensorflow and CUDA information
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print("Num CPUs Available: ", len(tf.config.experimental.list_physical_devices('CPU')))
print(f"Tensorflow version: {tf.__version__}")
print(f"Keras version: {keras.__version__}")

if tf.test.gpu_device_name():
    gpu_devices = tf.config.list_physical_devices('GPU')
    details = tf.config.experimental.get_device_details(gpu_devices[0])
    name = details.get('device_name', 'Unknown GPU')
    
    print(f"Using {name}")
else:
    print("No GPU found")


Num GPUs Available:  1
Num CPUs Available:  1
Tensorflow version: 2.10.1
Keras version: 2.10.0
Using NVIDIA GeForce RTX 3070 Laptop GPU


In [5]:
classes = [f for f in os.listdir('../Data/Birds/images') if not f.startswith('.')]

# find the smallest number of images in a catagory
min_images = 100000
for catagory in classes:
    num_images = len(os.listdir(f'../Data/Birds/images/{catagory}'))
    if num_images < min_images:
        min_images = num_images

print(f"Smallest number of images in a catagory: {min_images}")

images_per_catagory = min_images

# delete images from catagories with more than images_per_catagory images
# this part was only run once to make loading the data easier
# for catagory in catagories:
#    for file in os.listdir(f'../Data/Birds/images/{catagory}'):
#        # if the number of the file is greater than images_per_catagory
#        if int(file.split('.')[0]) >= images_per_catagory:
#            os.remove(f'../Data/Birds/images/{catagory}/{file}')

Smallest number of images in a catagory: 140


Here I have essentally dropped ~30,000 images from the dataset but the remaining size of the is still 63,000. Even after removing the excess data the dataset is still adequetely sized for an 80/20 split further down the line. As the data was gathered through google image searches this is not a signifigant loss of resources. This also comes with the benifit of balancing the dataset as the number of images of each class is now equal.

A straight forward usecase of a bird classifier could be an app that can tell people what bird they are looking at by taking a picture of it. For this use I think the best metric is accuracy as the value of correctly identifying a bird is the same as the value lost when incorrectly identifying one. 

In [6]:
height = 224
width = 224
channels = 3

print(f"No. of catagories: {len(classes)}")
num_classes = len(classes)

start = time.time()

# load each image into a (200 * catagories) x height x width x channels array
X = np.ndarray((len(classes) * images_per_catagory, height, width, channels), dtype=np.float32)
y = np.ndarray((len(classes) * images_per_catagory), dtype=np.uint16)

for i, catagory in enumerate(classes):
    # only load 200 images per catagory
    for j in range(images_per_catagory):
        file = f"{j}.jpg"
        try: 
            img = cv.imread(f'../Data/Birds/images/{catagory}/{file}')
            X[i * images_per_catagory + j] = img
            y[i * images_per_catagory + j] = i
        except Exception as e:
            print(f"Error loading {catagory}/{file}")
            print(e)
    print(f"Percentage complete: {i / len(classes) * 100:.2f}%", end='\r')

print(f"{len(X)} images loaded for {len(classes)} catagories in {time.time() - start:.2f} seconds")

# Normalise the colour values
np.divide(X, 255, out=X)

No. of catagories: 450
54000 images loaded for 450 catagories in 41.52 seconds


array([[[[0.5921569 , 0.62352943, 0.5803922 ],
         [0.5921569 , 0.62352943, 0.5803922 ],
         [0.6       , 0.6313726 , 0.5882353 ],
         ...,
         [0.17254902, 0.45490196, 0.40784314],
         [0.17254902, 0.45490196, 0.40784314],
         [0.17254902, 0.45490196, 0.40784314]],

        [[0.5921569 , 0.62352943, 0.5803922 ],
         [0.59607846, 0.627451  , 0.58431375],
         [0.6039216 , 0.63529414, 0.5921569 ],
         ...,
         [0.17254902, 0.45490196, 0.40784314],
         [0.17254902, 0.45490196, 0.40784314],
         [0.17254902, 0.45490196, 0.40784314]],

        [[0.60784316, 0.6313726 , 0.5882353 ],
         [0.6117647 , 0.63529414, 0.5921569 ],
         [0.61960787, 0.6431373 , 0.6       ],
         ...,
         [0.17254902, 0.45490196, 0.40784314],
         [0.17254902, 0.45490196, 0.40784314],
         [0.17254902, 0.45490196, 0.40784314]],

        ...,

        [[0.10980392, 0.3254902 , 0.2901961 ],
         [0.10980392, 0.3254902 , 0.2901961 ]

In [ ]:
sys.getsizeof(X)

# save X to a pickle file
with open('../Data/Pickle/birds_data.pickle', 'wb') as f:
    pickle.dump(X, f)

: 

: 

In [ ]:
print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")

Here the dataset is verified to have loaded 63,000 images each 224x244 and with 3 color channels

In [ ]:
def plot_random_gallery(images, labels, titles, h, w, n_row=3, n_col=6):
    """Helper function to plot a gallery of portraits"""
    plt.tight_layout()
    plt.figure(figsize=(1.7 * n_col, 2.3 * n_row))
    plt.subplots_adjust(bottom=0, left=.01, right=.99, top=.90, hspace=.35)
    for i in range(n_row * n_col):
        # get random image
        rand = np.random.randint(0, len(images))
        plt.subplot(n_row, n_col, i + 1)
        plt.imshow(images[rand])
        plt.title(titles[labels[rand]], size=10)
        plt.xticks(())
        plt.yticks(())
    plt.show()

plot_random_gallery(X, y, classes, height, width) # defaults to showing a 3 by 6 subset of the bird images


A sample of the birds in the dataset is shown above along with what type of bird is in each image

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, test_size=0.2, stratify=y)

# convert the labels to one-hot vectors
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# Normalise the data in each channel
X_train /= 255
X_test /= 255
X_train -= 0.5
X_test -= 0.5